In [69]:
import numpy as np
import matplotlib.pyplot as plt
import time
from scipy.fftpack import fft, fft2, ifft2
from scipy.integrate import solve_ivp
from scipy.linalg import solve, solve_triangular, lu
from scipy.sparse import spdiags

In [127]:
t_span = np.arange(0, 60.5, 0.5)
nu = 0.001
Lx, Ly = 20, 20
nx, ny = 64, 64
N = nx * ny

x2 = np.linspace(-Lx/2, Lx/2, nx + 1)
x = x2[:nx]
y2 = np.linspace(-Ly/2, Ly/2, ny + 1)
y = y2[:ny]
X, Y = np.meshgrid(x, y)

omega1 = (np.exp((-(X-5)**2 - Y**2 / 10)) - np.exp((-(X+5)**2 - Y**2 / 10))).flatten()
omega2 = (np.exp((-(X-5)**2 - Y**2 / 10)) + np.exp((-(X+5)**2 - Y**2 / 10))).flatten()
omega3 = (np.exp(-((X - 3)**2 + (Y - 3)**2)/4) - np.exp(-((X + 3)**2 + (Y - 3)**2)/4)
                      - np.exp(-((X - 3)**2 + (Y + 3)**2)/4) + np.exp(-((X + 3)**2 + (Y + 3)**2)/4)).flatten()
omega4 = np.zeros_like(X)
np.random.seed(50)  # For reproducibility
for _ in range(12):
    x0, y0 = np.random.uniform(-6, 6,size=2)
    strength = np.random.choice([1, -1]) * np.random.uniform(1, 3)
    omega4 += strength * np.exp(-((X - x0)**2 + (Y - y0)**2)/4)

omega4 = omega4.flatten()
print(np.size(omega3))
print(np.size(omega4))


kx = (2 * np.pi / Lx) * np.concatenate((np.arange(0, nx/2), np.arange(-nx/2, 0)))
kx[0] = 1e-6
ky = (2 * np.pi / Ly) * np.concatenate((np.arange(0, ny/2), np.arange(-ny/2, 0)))
ky[0] = 1e-6
KX, KY = np.meshgrid(kx, ky)
K = KX**2 + KY**2

4096
4096


In [113]:
# HW4 i.c.
m = 64
L = 20
dx = L / m
n = m * m

e0 = np.zeros(n)
e1 = np.ones(n)
e2 = np.copy(e1)
e4 = np.copy(e0)

for j in range(1, m+1):
    e2[m*j - 1] = 0 
    e4[m*j - 1] = 1

# Adjusted vectors for diagonals
e3 = np.zeros_like(e2)
e3[1:n] = e2[0:n-1]
e3[0] = e2[n-1]

e5 = np.zeros_like(e4)
e5[1:n] = e4[0:n-1]
e5[0] = e4[n-1]

# Construct Matrix a (Laplacian)
diagonals_A = [e1, e1, e5, e2, -4 * e1, e3, e4, e1, e1]
offsets_A = [-(n - m), -m, -m + 1, -1, 0, 1, m - 1, m, (n - m)]
A = (spdiags(diagonals_A, offsets_A, n, n) / (dx**2)).toarray()

# Construct Matrix C (Partial derivative with respect to y)
diagonals_B = [e1, -e1, e1, -e1]
offsets_B = [-(n - m),-m, m, (n - m)]
B = (spdiags(diagonals_B, offsets_B, n, n) / (2 * dx)).toarray()

# Construct Matrix b (Partial derivative with respect to x)
diagonals_C = [e5, -e2, e3, -e4]
offsets_C = [-m + 1, -1, 1, m - 1]
C = (spdiags(diagonals_C, offsets_C, n, n) / (2 * dx)).toarray()

A[0, 0] = 2

In [128]:
def spc_rhs(t, omega, nx, ny, K, nu):
    wt = fft2(omega.reshape((nx, ny)))
    psit = -wt / K
    psi = (np.real(ifft2(psit))).flatten()
    rhs = nu * np.dot(A, omega) - np.dot(B, psi) * np.dot(C, omega) + np.dot(C, psi) * np.dot(B, omega)
    return rhs


wtsol1 = solve_ivp(spc_rhs, (0, t_span[-1]), omega1, t_eval=t_span, method = 'RK45', args = (nx, ny, K, nu))
wtsol2 = solve_ivp(spc_rhs, (0, t_span[-1]), omega2, t_eval=t_span, method = 'RK45', args = (nx, ny, K, nu))
wtsol3 = solve_ivp(spc_rhs, (0, t_span[-1]), omega3, t_eval=t_span, method = 'RK45', args = (nx, ny, K, nu))
wtsol4 = solve_ivp(spc_rhs, (0, t_span[-1]), omega4, t_eval=t_span, method = 'RK45', args = (nx, ny, K, nu))
A1 = wtsol1.y
A2 = wtsol2.y
A3 = wtsol3.y
A4 = wtsol4.y



In [129]:
# solver for oppsite amps
import imageio.v2 as imageio
import os

gif_frames = []

# # Plot the solution at each time step and save frames
for j, t in enumerate(t_span):
    wtc_opp = A1[:, j].reshape((ny, nx)) 
    
    # Create plot
    fig, ax = plt.subplots()
    c = ax.contourf(x, y, wtc_opp, levels=50, cmap='plasma')
    fig.colorbar(c, ax=ax)
    ax.set_title(f'Time: {t}')
    
    # Save the current plot as an image frame
    plt.savefig('frame.png')  # Save frame to file
    plt.close(fig)
    
    # Append the frame to gif_frames list
    gif_frames.append(imageio.imread('frame.png'))

# Create the .gif
imageio.mimsave('opposite_charged.gif', gif_frames, duration=0.2)  # 0.5 seconds between frames
os.remove('frame.png')

print("GIF created successfully!")

GIF created successfully!


In [130]:
# solver for same amps
import imageio.v2 as imageio
import os

gif_frames = []

# # Plot the solution at each time step and save frames
for j, t in enumerate(t_span):
    wtc_same = A2[:, j].reshape((ny, nx))
    
    # Create plot
    fig, ax = plt.subplots()
    c = ax.contourf(x, y, wtc_same, levels=50, cmap='plasma')
    fig.colorbar(c)
    ax.set_title(f'Time: {t}')
    
    # Save the current plot as an image frame
    plt.savefig('frame.png')  # Save frame to file
    plt.close(fig)
    
    # Append the frame to gif_frames list
    gif_frames.append(imageio.imread('frame.png'))

# Create the .gif
imageio.mimsave('same_charge.gif', gif_frames, duration=0.2)  # 0.5 seconds between frames
os.remove('frame.png')

print("GIF created successfully!")

GIF created successfully!


In [131]:
# solver for oppsite amps colliding
import imageio.v2 as imageio
import os

gif_frames = []

# # Plot the solution at each time step and save frames
for j, t in enumerate(t_span):
    wtc_opp_collide = A3[:, j].reshape((ny, nx)) 
    
    # Create plot
    fig, ax = plt.subplots()
    c = ax.contourf(x, y, wtc_opp_collide, levels=50, cmap='plasma')
    fig.colorbar(c)
    ax.set_title(f'Time: {t}')
    
    # Save the current plot as an image frame
    plt.savefig('frame.png')  # Save frame to file
    plt.close(fig)
    
    # Append the frame to gif_frames list
    gif_frames.append(imageio.imread('frame.png'))

# Create the .gif
imageio.mimsave('opposite_collide.gif', gif_frames, duration=0.2)  # 0.5 seconds between frames
os.remove('frame.png')

print("GIF created successfully!")

GIF created successfully!


In [132]:
# solver for random assortment of 12 vortices

import imageio.v2 as imageio
import os

gif_frames = []

# # Plot the solution at each time step and save frames
for j, t in enumerate(t_span):
    wtc_random = A4[:, j].reshape((ny, nx)) 
    
    # Create plot
    fig, ax = plt.subplots()
    c = ax.contourf(x, y, wtc_random, levels=50, cmap='plasma')
    fig.colorbar(c)
    ax.set_title(f'Time: {t}')
    
    # Save the current plot as an image frame
    plt.savefig('frame.png')  # Save frame to file
    plt.close(fig)
    
    # Append the frame to gif_frames list
    gif_frames.append(imageio.imread('frame.png'))

# Create the .gif
imageio.mimsave('random_assortment.gif', gif_frames, duration=0.2)  # 0.5 seconds between frames
os.remove('frame.png')

print("GIF created successfully!")

GIF created successfully!
